In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [134]:
#cd = webdriver.Chrome('C:\\Users\김성제\Downloads\WinPython-64bit-3.5.2.2\chromedriver_win32\\chromedriver.exe')

In [25]:
person_url = "https://www.1365.go.kr/vols/P9210/partcptn/timeCptn.do?cPage={0}" # 개인봉사참여, 추후 formatting
org_url = 'https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?cPage={0}' # 단체봉사참여, 추후 formatting

# 상세정보 페이지
detail_page = "https://www.1365.go.kr/vols/P9210/partcptn/timeCptn.do\
?titleNm=%EC%83%81%EC%84%B8%EB%B3%B4%EA%B8%B0&type=show&progrmRegistNo={0}" # 개인봉사 상세정보 페이지
detail_page_org = 'https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do\
?titleNm=상세보기&type=show&progrmRegistNo={0}' # 단체봉사 상세정보 페이지

headers = {'accept-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.183 Safari/537.36 Vivaldi/1.96.1147.55'}

In [6]:
resp = requests.get(org_url, headers = headers)
html = BeautifulSoup(resp.text, 'lxml')

In [7]:
# javascript link id 뽑기
link_IDs = []
links = html.select('#content > div.content_view > div > div.board_list.data_middle > ul > li > input[value]')
for link in links: 
    link_IDs.append(link.attrs['value'])

In [10]:
# link id를 이용하여 각 페이지마다 긁어오기
for link_num in link_IDs:
    resp_detail = requests.get(detail_page.format(link_num), headers = headers)
    html_detail = BeautifulSoup(resp_detail.text, 'lxml')
    sentence = html_detail.select('#content > div.content_view > div > div.board_view.type2 > h3')[0].text
    print(" ".join(sentence.split())) # 굉장히 개쩌는 유용한 스킬

제3세계 어린이를 위한 비누제작 및 기부 (모집중)
학습지도및 놀이지도 (모집중)
식사보조, 말벗, 청소, 이동보조(고등학생이상만 가능) (모집중)
아름다운가게 통영중앙점 자원봉사자 모집(3개월 이상 주1회 정기봉사자) (모집중)


In [26]:
crawl_dict = {
    # 제목
     'vol_title' : " ".join(html_detail.select('#content > div.content_view > div > \
     div.board_view.type2 > h3')[0].text.split()),
    
    # 봉사기간
    'vol_period' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[0].text,

    # 봉사시간
    'vol_time' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[1].text.split()),

    # 모집기간
    'vol_period_gath' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[2].text,

    # 모집인원
    'vol_applicant' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[3].text,

    # 활동요일
    'vol_act_day' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[4].text.split()),

    # 신청인원수
    'vol_apply' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[5].text.replace("명", ""),

    # 봉사분야
    'vol_field' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[6].text,

    # 봉사자유형
    'vol_char' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[7].text.split()),

    # 모집기관
    'vol_institution' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[8].text.split()),

    # 등록기관
    'vol_administrator' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[9].text.split()),

    # 봉사장소
    'vol_place' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[10].text.split()),

    # 봉사대상
    'vol_target' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
    > div.board_data.type2 > div.group > dl > dd')[11].text.split()),

    #####
    # 내용
    'vol_describe' : html_detail.select('#content > div.content_view > div > div.board_view.type2 > \
    div.board_body > div > pre')[0].text,
    #####

    # 담당자명
    'vol_director' : html_detail.select('#content > div.content_view > div > div.board_view.type2 > \
    div.incharge_data > dl.name > dd')[0].text.replace(": ", ""),

    # 전화번호
    'vol_call' : html_detail.select('#content > div.content_view > div > \
    div.board_view.type2 > div.incharge_data > dl.tel > dd')[0].text.replace(": ", ""),

    # FAX
    'vol_fax' : html_detail.select('#content > div.content_view > div > \
    div.board_view.type2 > div.incharge_data > dl.fax > dd')[0].text.replace(": ", ""),

    # 주소
    'vol_address' : html_detail.select('#content > div.content_view > div > div.board_view.type2 > \
    div.incharge_data > dl.addr > dd')[0].text.replace(": ", "")

}

In [229]:
aa = []
for k, v in crawl_dict.items():
    aa.append(v)

In [29]:
col_name= ['제목', '봉사기간', '봉사시간', '모집기간', '모집인원', '활동요일', '신청인원수',\
            '봉사분야', '봉사자유형', '모집기관', '등록기관', '봉사장소', '봉사대상', '내용',\
            '담당자명', '전화번호', 'FAX', '주소']

In [263]:
count = 0
list_1365 = pd.DataFrame(columns = col_name)

In [247]:
list_1365.loc[count] = aa
count += 1

---

## 개인 봉사

In [291]:
count = 0
list_1365 = pd.DataFrame(columns = col_name)

In [284]:
import pickle

In [288]:
with open(r'C:\Users\김성제\Desktop\link_IDs.txt',"wb") as fp :
    pickle.dump(link_IDs,fp)

In [289]:
with open(r'C:\Users\김성제\Desktop\link_IDs.txt',"rb") as fp :
    test = pickle.load(fp)

In [298]:
detail_page.format(link_num)

'https://www.1365.go.kr/vols/P9210/partcptn/timeCptn.do?titleNm=%EC%83%81%EC%84%B8%EB%B3%B4%EA%B8%B0&type=show&progrmRegistNo=2329752'

In [ ]:
link_IDs = []

for i in range(1, len(link_IDs)):
    resp = requests.get(person_url.format(i), headers = headers)
    html = BeautifulSoup(resp.text, 'lxml')
    #time.sleep(3)

    links = html.select('#content > div.content_view > div > div.board_list.data_middle > ul > li > input[value]')
    for link in links: 
        link_IDs.append(link.attrs['value'])

In [ ]:
try:
    for link_num in link_IDs:
        resp_detail = requests.get(detail_page.format(link_num), headers = headers)
        html_detail = BeautifulSoup(resp_detail.text, 'lxml')
        time.sleep(1)

        crawl_dict = {
        # 제목
         'vol_title' : " ".join(html_detail.select('#content > div.content_view > div > \
         div.board_view.type2 > h3')[0].text.split()),

        # 봉사기간
        'vol_period' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[0].text,

        # 봉사시간
        'vol_time' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[1].text.split()),

        # 모집기간
        'vol_period_gath' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[2].text,

        # 모집인원
        'vol_applicant' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[3].text,

        # 활동요일
        'vol_act_day' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[4].text.split()),

        # 신청인원수
        'vol_apply' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[5].text.replace("명", ""),

        # 봉사분야
        'vol_field' : html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[6].text,

        # 봉사자유형
        'vol_char' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[7].text.split()),

        # 모집기관
        'vol_institution' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[8].text.split()),

        # 등록기관
        'vol_administrator' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[9].text.split()),

        # 봉사장소
        'vol_place' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[10].text.split()),

        # 봉사대상
        'vol_target' : " ".join(html_detail.select('#content > div.content_view > div > div.board_view.type2 \
        > div.board_data.type2 > div.group > dl > dd')[11].text.split()),

        #####
        # 내용
        'vol_describe' : html_detail.select('#content > div.content_view > div > div.board_view.type2 > \
        div.board_body > div > pre')[0].text,
        #####

        # 담당자명
        'vol_director' : html_detail.select('#content > div.content_view > div > div.board_view.type2 > \
        div.incharge_data > dl.name > dd')[0].text.replace(": ", ""),

        # 전화번호
        'vol_call' : html_detail.select('#content > div.content_view > div > \
        div.board_view.type2 > div.incharge_data > dl.tel > dd')[0].text.replace(": ", ""),

        # FAX
        'vol_fax' : html_detail.select('#content > div.content_view > div > \
        div.board_view.type2 > div.incharge_data > dl.fax > dd')[0].text.replace(": ", ""),

        # 주소
        'vol_address' : html_detail.select('#content > div.content_view > div > div.board_view.type2 > \
        div.incharge_data > dl.addr > dd')[0].text.replace(": ", "")

        }

        aa = []
        for v in crawl_dict.values():
            aa.append(v)
        list_1365.loc[count] = aa
        count += 1
        print(count)
except:
    print("error ", count)
    count += 1
    pass

In [312]:
with open(r'C:\Users\김성제\Desktop\list_1365.txt',"wb") as fp :
    pickle.dump(list_1365,fp)

In [317]:
list_1365.to_csv("list_1365.csv", mode='w', encoding = 'utf-8-sig')

---

## 단체 봉사

In [64]:
col_name= ['제목', '봉사기간', '봉사시간', '모집기간', '모집인원', '활동요일', '신청인원수', \
            '봉사분야', '봉사자유형', '모집기관', '등록기관', '봉사장소', '봉사대상', '내용', \
            '담당자명', '전화번호', 'FAX', '주소']

In [67]:
count = 0
list_1365_org = pd.DataFrame(columns = col_name)

In [6]:
link_IDs = []
for i in range(1, 104):
    resp = requests.get(org_url.format(i), headers = headers)
    html = BeautifulSoup(resp.text, 'lxml')
    time.sleep(2)
 
    links = html.select('#content > div.content_view > div > div.board_list.data_middle > ul > li > input[value]')
    for link in links: 
        link_IDs.append(link.attrs['value'])
    if i % 10 == 0:
        print(i) 

10
20
30
40
50
60
70
80
90
100


In [11]:
import pickle

In [23]:
with open(r'C:\Users\김성제\Desktop\list_1365_org_link.txt',"wb") as fp :
    pickle.dump(link_IDs, fp)

In [32]:
resp_detail = requests.get(detail_page_org.format(link_num), headers = headers)

In [33]:
html_detail = BeautifulSoup(resp_detail.text, 'lxml')

In [68]:
for link_num in link_IDs:
    try:
        resp_detail = requests.get(detail_page_org.format(link_num), headers = headers)
        print(detail_page_org.format(link_num))
        html_detail = BeautifulSoup(resp_detail.text, 'lxml')
        time.sleep(1)

        crawl_dict_org = {
            
        # 제목
         'vol_title' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > h3')[0].text.split()),

        # 봉사기간
        'vol_period' : html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[0].text,
        # 봉사시간
        'vol_time' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[1].text.split()),    

        # 모집기간
        'vol_period_gath' : html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[2].text,

        # 모집인원
        'vol_applicant' : html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[3].text,
        # 활동요일
        'vol_act_day' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[4].text.split()),

        # 신청인원수
        'vol_apply' : html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[5].text.replace("명", ""),

        # 봉사분야
        'vol_field' : html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[6].text,

        # 봉사자유형
        'vol_char' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[7].text.split()),

        # 모집기관
        'vol_institution' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[8].text.split()),

        # 등록기관
        'vol_administrator' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[9].text.split()),

        # 봉사장소
        'vol_place' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[10].text.split()),

        # 봉사대상
        'vol_target' : " ".join(html_detail.select('#frm > div > div.board_view.type2 > \
                                          div.board_data.type2 > div.group > dl > dd')[11].text.split()),
                
        ##### 내용
        'vol_describe' : html_detail.select('#frm > div > div.board_view.type2 > div.board_body > div > pre')[0].text,
        #####

        # 담당자명
        'vol_director' : html_detail.select('#frm > div > div.board_view.type2 > \
        div.incharge_data > dl.name > dd')[0].text.replace(": ", ""),

        # 전화번호
        'vol_call' : html_detail.select('#frm > div > div.board_view.type2 > \
        div.incharge_data > dl.tel > dd')[0].text.replace(": ", ""),

        # FAX
        'vol_fax' : html_detail.select('#frm > div > div.board_view.type2 > \
        div.incharge_data > dl.fax > dd')[0].text.replace(": ", ""),

        # 주소
        'vol_address' : html_detail.select('#frm > div > div.board_view.type2 > \
        div.incharge_data > dl.addr > dd')[0].text.replace(": ", "")

        }
        aa = []
        for v in crawl_dict_org.values():
            aa.append(v)
        list_1365_org.loc[count] = aa
        count += 1
        print(count)
        list_1365_org.to_csv("list_1365_org.csv", mode='w', encoding = 'utf-8-sig')
    except:
        print("error ", count)
        count += 1
        pass

https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283294
1
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283296
2
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283300
3
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283305
4
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283306
5
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283310
6
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283312
7
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283314
8
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progrmRegistNo=2283537
9
https://www.1365.go.kr/vols/P9230/partcptn/grpCptn.do?titleNm=상세보기&type=show&progr